In [1]:
import matplotlib; matplotlib.use("tkagg")
from matplotlib import animation
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [2]:
class CrossEntropy:
    """
    Optimize a function using the Cross Entropy method. Adapted from:
    http://www.cleveralgorithms.com/nature-inspired/probabilistic/cross_entropy.html
    """

    CURVE_DENSITY = 40


    def __init__(self, dimension: int, bounds: np.ndarray, max_iterations: int, num_samples: int, num_update: int, learning_rate: float):
        """
        Initialize parameters for Cross Entropy method of optimization.

        :param dimension: number of dimensions to optimize.
        :param bounds: list of tuples of upper/lower bounds. List is the same size as dimension.
        :param max_iterations: maximum number of iterations in performing optimization.
        :param num_samples: number of samples in Monte Carlo simulation.
        :param num_update: number of samples to consider when updating distribution.
        :param learning_rate: speed of convergence.
        """

        self.dimension = dimension
        self.bounds = bounds.astype(np.float64)
        self.max_iterations = max_iterations
        self.num_samples = num_samples
        self.num_update = num_update
        self.learning_rate = learning_rate


    def optimize(self, objective_function: callable) -> (np.ndarray, list):
        """
        Optimizes given objective function.

        :param objective_function: function to optimize
        :param plot: whether to plot
        :return: best solution, list of individuals by iteration
        """

        # Randomly initialize means within given bounds
        means = self.bounds[:, 0] + np.multiply(
            np.random.rand(self.dimension),
            self.bounds[:, 1] - self.bounds[:, 0]
        )

        # Initialize standard deviations as range of bounds
        standard_deviations = self.bounds[:, 1] - self.bounds[:, 0]
        standard_deviations = standard_deviations.astype(np.float64)

        # Initialize current best to None
        best_individual = None
        best_cost = None
        best_gen = None

        samples_by_iteration = []

        # Iterate until convergence or until max iterations is reached
        for iteration in range(self.max_iterations):

            # Generate samples from a Gaussian distribution between bounds
            samples = np.asarray(
                [[max(min(np.random.normal(means[i], standard_deviations[i]), self.bounds[i, 1]), self.bounds[i, 0]) for i in range(self.dimension)] for _ in range(self.num_samples)]
            )
            sample_costs = objective_function(samples)

            # Sort samples and sample costs by index
            sorted_indices = sample_costs.argsort()
            samples = samples[sorted_indices]
            sample_costs = sample_costs[sorted_indices]

            # Update best individual if we have discovered a new best
            if best_individual is None or best_cost > sample_costs[0]:
                best_individual, best_cost = samples[0], sample_costs[0]
                best_gen = iteration

            # Select individuals to influence distribution
            selected_individuals = samples[:self.num_update]

            # Update distribution parameters
            for i in range(self.dimension):
                means[i] = self.learning_rate * means[i] + ((1.0-self.learning_rate) * np.mean(selected_individuals[:, i]))
                standard_deviations[i] = self.learning_rate * standard_deviations[i] + ((1.0-self.learning_rate) * np.std(selected_individuals[:, i]))

            # Display status of algorithm
            print("Iteration {iteration}\t\tBest cost: {fitness}\t\tBest individual: {individual}".format(
                iteration=str.zfill(str(iteration), 3),
                fitness=repr(best_cost),
                individual=", ".join(list(map(str, best_individual)))
            ))

            samples_by_iteration.append(samples)

        return best_individual, samples_by_iteration

In [3]:
def Rastrigin(x: np.ndarray, A: float = 10, n: int = 2) -> float:
    """
    Benchmark function with many local optima.
    Global minimum is at x = 0.
    Bounds: (-5.12, 5.12)

    :param x: input vector
    :param A: arbitrary constant, typically A=10
    :param n: number of dimensions
    :return: value of Rastrigin function
    """
    total = np.ones((x.shape[0]))*A*n
    for i in range(n):
        total += (np.power(x[:, i], 2) - A*np.cos(2*np.pi*x[:, i]))
    return total.T

In [4]:
def Ackley(x: np.ndarray) -> float:
    """
    Ackley function for benchmarking.
    Global minimum at (0, 0).
    Bounds: (-5, 5)

    :param x: input vector, must be 2D
    :return: value of Ackley function
    """
    return -20*np.exp(-0.2*np.sqrt(0.5*(np.power(x[:, 0], 2) + np.power(x[:, 1], 2)))) - \
        np.exp(0.5*(np.cos(2*np.pi*x[:, 0]) + np.cos(2*np.pi*x[:, 1]))) + np.e + 20

In [5]:
def Easom(x: np.ndarray) -> float:
    """
    Easom function for benchmarking.
    Global minimum at f(pi, pi) = -1
    Bounds: (-100, 100)

    :param x: input vector, must be 2D
    :return: value of Easom function
    """
    return -np.cos(x[:, 0])*np.cos(x[:, 1])*np.exp(-(np.power(x[:, 0] - np.pi, 2) + np.power(x[:, 1] - np.pi, 2)))


In [6]:
def Cross_in_Tray(x: np.ndarray) -> float:
    """
    Cross-in-tray function for benchmarking.
    Global minimums at f(+- 1.34941, +-1.34941) = -2.06261
    Bounds: (-10, 10)

    :param x: input vector, must be 2D
    :return: value of Cross_in_Tray function
    """
    return -0.0001*np.power(np.abs(np.sin(x[:, 0])*np.sin(x[:, 1])*np.exp(np.abs(100-np.sqrt(np.power(x[:, 0], 2) + np.power(x[:, 1], 2))/np.pi))) + 1, 0.1)


In [7]:
def Levi(x: np.ndarray) -> float:
    """
    Levi function for benchmarking.
    Global minimum at f(0, 0) = 0
    Bounds: (-5, 5)
    
    :param x: input vector, must be 2D
    :return: value of Levi function
    """
    return np.power(np.sin(3*np.pi*x[:, 0]), 2) + np.multiply(np.power(x[:, 0] - 1, 2), 1+np.power(np.sin(3*np.pi*x[:, 1]), 2))+np.multiply(np.power(x[:, 1]-1, 2), 1+np.power(np.sin(2*np.pi*x[:, 1]), 2))

In [8]:
def animate(objective_function: callable, samples_by_iteration, bounds: np.ndarray, curve_density: int):

    # Precalculate surface points for plotting
    xs = np.linspace(bounds[0][0], bounds[0][1], curve_density).reshape(
        (1, curve_density))
    ys = np.linspace(bounds[1][0], bounds[1][1], curve_density).reshape(
        (1, curve_density))
    X, Y = np.meshgrid(xs, ys)
    Z = objective_function(np.vstack((np.ravel(X), np.ravel(Y))).T).reshape(X.shape)

    fig = plt.figure()
    samples = fig.add_subplot(111, projection="3d")
    samples.plot_surface(X, Y, Z, color=(1, 1, 0, 0.5))
    individuals = samples.scatter(samples_by_iteration[0][:, 0], samples_by_iteration[0][:, 1], objective_function(samples_by_iteration[0]), c=(0, 0, 0), s=100)


    def init():
        return samples,

    def iterate(i):
        nonlocal individuals
        individuals.remove()
        individuals = samples.scatter(samples_by_iteration[i][:, 0], samples_by_iteration[i][:, 1], objective_function(samples_by_iteration[i]), c=(0, 0, 0), s=100)
        return samples,

    anim = animation.FuncAnimation(fig, iterate, init_func=init, frames=len(samples_by_iteration), interval=40, blit=False)
    plt.show()

In [ ]:
CE = CrossEntropy(
    dimension = 2,
    bounds = np.asarray([[-5, 5], [-5, 5]]),
    max_iterations = 150,
    num_samples = 50,
    num_update = 5,
    learning_rate = 0.9
)

#objective_function = lambda x : np.sum((np.power(x, 2)), axis=1); CE.bounds = np.asarray([[-5, 5], [-5, 5]])
#objective_function = Rastrigin; CE.bounds = np.asarray([[-5, 5], [-5, 5]])
#objective_function = Ackley; CE.bounds = np.asarray([[-5.12, 5.12], [-5.12, 5.12]])
#objective_function = Easom; CE.bounds = np.asarray([[-100, 100], [-100, 100]])
#objective_function = Cross_in_Tray; CE.bounds = np.asarray([[-10, 10], [-10, 10]])
objective_function = Levi; CE.bounds = np.asarray([[-5, 5], [-5, 5]])
best, samples_by_iteration = CE.optimize(objective_function)

animate(objective_function, samples_by_iteration, CE.bounds, CrossEntropy.CURVE_DENSITY)

Iteration 000		Best cost: 12.492814901167176		Best individual: -1.62640841984, -0.268661814627
Iteration 001		Best cost: 0.52089330988956317		Best individual: 0.626425799111, 1.38551547872
Iteration 002		Best cost: 0.52089330988956317		Best individual: 0.626425799111, 1.38551547872
Iteration 003		Best cost: 0.52089330988956317		Best individual: 0.626425799111, 1.38551547872
Iteration 004		Best cost: 0.52089330988956317		Best individual: 0.626425799111, 1.38551547872
Iteration 005		Best cost: 0.52089330988956317		Best individual: 0.626425799111, 1.38551547872
Iteration 006		Best cost: 0.52089330988956317		Best individual: 0.626425799111, 1.38551547872
Iteration 007		Best cost: 0.52089330988956317		Best individual: 0.626425799111, 1.38551547872
Iteration 008		Best cost: 0.52089330988956317		Best individual: 0.626425799111, 1.38551547872
Iteration 009		Best cost: 0.51164620893992574		Best individual: 1.25651002454, 1.03242534108
Iteration 010		Best cost: 0.51164620893992574		Best individu